# **0. Judul & Ringkasan**
**GTWR-GNN End-to-End (Paper-Style IPYNB)** — dari dasar teori (OLS, VCM, GWR/GTWR) hingga implementasi penuh (graf spasio-temporal, GNN untuk bobot, ridge-WLS untuk koefisien lokal, training semi-supervised, evaluasi). Semua dalam Bahasa Indonesia.

## **1. Pendahuluan**
Motivasi: heterogenitas spasial-temporal menyebabkan koefisien regresi berbeda menurut lokasi dan waktu. Model global (OLS) sering tidak cukup. GWR/GTWR memakai pembobotan lokal; GTWR-GNN mempelajari bobot adaptif dengan GNN.

### **1.1 Kontribusi & Alur**
- (i) OLS → VCM → GWR/GTWR (dasar teori)
- (ii) Hubungkan ke GNNWR, SRGCNN, GSTRGCN (state-of-the-art)
- (iii) Implementasi penuh: data → panel → graf → GNN → bobot → ridge-WLS → evaluasi
- (iv) Training semi-supervised (opsional) + perbaikan error reindex duplicate labels

## **2. Teori Dasar**
### **2.1 Regresi Linear (OLS)**
Model: y = X beta + eps, dengan E[eps|X]=0. Estimator: beta_hat = (X'X)^{-1}X'y.

### **2.2 Varying Coefficient Model (VCM)**
Koefisien beta_k(z) berubah halus terhadap variabel indeks z (lokasi/waktu). Estimasi lokal via pembobotan kernel.

### **2.3 GWR/GTWR**
GWR: z=(u,v) (spasial). GTWR: z=(u,v,t) (spasio-temporal). Estimasi lokal dengan bobot w_{(i,t),(j,s)} dan ridge untuk stabilitas.

## **3. Keterbatasan GTWR Klasik**
Bobot kernel fixed (bergantung jarak) kurang adaptif untuk relasi kompleks. Perlu bobot yang dipelajari dari data.

## **4. Keterkaitan Studi Terdahulu**
- **GNNWR/GTNNWR**: neural network untuk pembobotan regresi (spasial → spasio-temporal)
- **SRGCNN**: GCN untuk regresi spasial (membangun graf, message passing)
- **GSTRGCN**: kerangka umum spasio-temporal (GCN untuk spasial, Transformer untuk temporal)

## **5. Formulasi GTWR-GNN**
Model lokal: y_{i,t} = x_{i,t}' beta_{i,t} + eps_{i,t}. Bobot W dipelajari dari embedding GNN atas graf spasio-temporal. Estimator lokal di node q: beta_hat(q) = argmin_beta sum_r W[q,r] (y_r - x_r' beta)^2 + lambda ||beta||^2.

In [1]:

# 6. Implementasi: Import & Konfigurasi
import os, warnings, random, math
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

try:
    from torch_geometric.nn import GCNConv, GATv2Conv
    HAS_PYG = True
except Exception:
    HAS_PYG = False

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings("ignore")
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

# Konfigurasi data
PATH_XLSX = "Data BPS Laporan KP - Coded.xlsx"  # ubah jika ada file
LAT_COL, LON_COL = "lat", "lon"
TIME_COL, TARGET_COL = "Tahun", "y"
FEATURE_COLS = ["X1","X2","X3","X4","X5","X6","X7","X8"]

# Parameter model
K_SPATIAL = 8
USE_TEMPORAL = True
RIDGE_LAMBDA = 0.1

# Fallback jika file tidak ada
USE_SYNTHETIC_IF_MISSING = True
N_GRID_X, N_GRID_Y = 8, 8  # grid lokasi sintetis per waktu
T_SYN = 5
P_SYN = len(FEATURE_COLS)


Device: cpu


In [2]:

# 7.1 Loader Data (dengan fallback sintetis)
def make_synthetic_panel(n_x=8, n_y=8, T=5, p=8, seed=42):
    rng = np.random.default_rng(seed)
    xs = np.linspace(-6.5, -6.0, n_x)
    ys = np.linspace(106.7, 107.2, n_y)
    coords = [(x,y) for x in xs for y in ys]
    rows = []
    beta0 = rng.normal(0.0, 0.5, size=(n_x*n_y, p))
    for t in range(2019, 2019+T):
        drift = 0.2*(t-2019)
        for i,(lat,lon) in enumerate(coords):
            x = rng.normal(0,1,size=p)
            y = x @ (beta0[i] + 0.1*drift) + 0.3*lat + 0.1*lon + rng.normal(0,0.5)
            row = {"lat": lat, "lon": lon, "Tahun": t, "y": y}
            for k in range(p):
                row[f"X{k+1}"] = x[k]
            rows.append(row)
    return pd.DataFrame(rows)

def load_or_make_dataframe(path_xlsx):
    if os.path.exists(path_xlsx):
        try:
            df = pd.read_excel(path_xlsx)
            print("Loaded:", path_xlsx, "shape:", df.shape)
            return df
        except Exception as e:
            print("Read error, using synthetic. Error:", e)
    if USE_SYNTHETIC_IF_MISSING:
        df = make_synthetic_panel(n_x=N_GRID_X, n_y=N_GRID_Y, T=T_SYN, p=len(FEATURE_COLS), seed=SEED)
        print("Using synthetic dataset. shape:", df.shape)
        return df
    raise FileNotFoundError("Data file not found and synthetic disabled.")


In [3]:

# 7.2 Panel Builder (memperbaiki duplicate labels pada reindex)
def build_panel_arrays(df, time_col, target_col, feature_cols, lat_col, lon_col, times_sorted):
    df = df.copy()
    # pastikan kolom ada
    needed = set([time_col, target_col, lat_col, lon_col] + feature_cols)
    missing = needed - set(df.columns)
    if missing:
        raise ValueError(f"Missing columns: {missing}")
    # key lokasi stabil
    df["coord_key"] = list(zip(df[lat_col].round(6), df[lon_col].round(6)))
    # buang duplikat (lokasi-waktu)
    df = df.drop_duplicates(subset=["coord_key", time_col])
    # agregasi jika masih ada multiple rows
    df = df.groupby(["coord_key", time_col], as_index=False).mean(numeric_only=True)

    key_list = sorted(df["coord_key"].unique())

    X_blocks, y_blocks, coords_blocks = [], [], []
    for t in times_sorted:
        block = df[df[time_col] == t].copy().set_index("coord_key")
        if not block.index.is_unique:
            block = block[~block.index.duplicated(keep="first")]
        block = block.reindex(key_list).reset_index(drop=False)
        X_blocks.append(block[feature_cols].values.astype(np.float32))
        y_blocks.append(block[target_col].values.astype(np.float32).reshape(-1,1))
        coords_blocks.append(block[[lat_col, lon_col]].values.astype(np.float32))

    X_all = np.stack(X_blocks, axis=0)   # [T,N,p]
    y_all = np.stack(y_blocks, axis=0)   # [T,N,1]
    coords_blocks = np.stack(coords_blocks, axis=0) # [T,N,2]
    N_per_year = X_blocks[0].shape[0]
    return {"X_all": X_all, "y_all": y_all, "coords_blocks": coords_blocks,
            "times": np.array(times_sorted), "N_per_year": N_per_year, "key_list": key_list}


In [4]:

# 8.1 Konstruksi Graf Spasio-Temporal
def build_spatiotemporal_graph(coords_blocks, k_spatial=8, use_temporal=True):
    T, N, _ = coords_blocks.shape
    rows, cols = [], []
    # Spatial per waktu
    for ti in range(T):
        coords = coords_blocks[ti]
        A = kneighbors_graph(coords, k_spatial, mode="connectivity", include_self=False)
        Ai = A.tocoo()
        off = ti * N
        rows.extend((Ai.row + off).tolist())
        cols.extend((Ai.col + off).tolist())
    # Temporal links
    if use_temporal:
        for ti in range(T-1):
            off_curr = ti * N
            off_next = (ti+1) * N
            rows.extend([off_curr + i for i in range(N)]); cols.extend([off_next + i for i in range(N)])
            rows.extend([off_next + i for i in range(N)]); cols.extend([off_curr + i for i in range(N)])
    edge_index = np.vstack([np.array(rows, dtype=np.int64), np.array(cols, dtype=np.int64)])
    return edge_index


In [5]:

# 8.2 Encoder GNN sederhana (GCN/GAT jika tersedia)
class SimpleGCN(nn.Module):
    def __init__(self, in_dim, hidden_dim=64, out_dim=32, use_gat=False):
        super().__init__()
        self.use_gat = use_gat and HAS_PYG
        if HAS_PYG:
            if self.use_gat:
                self.conv1 = GATv2Conv(in_dim, hidden_dim, heads=1, dropout=0.0)
                self.conv2 = GATv2Conv(hidden_dim, out_dim, heads=1, dropout=0.0)
            else:
                self.conv1 = GCNConv(in_dim, hidden_dim)
                self.conv2 = GCNConv(hidden_dim, out_dim)
        else:
            self.fc1 = nn.Linear(in_dim, hidden_dim)
            self.fc2 = nn.Linear(hidden_dim, out_dim)

    def forward(self, x, edge_index=None):
        if HAS_PYG:
            h = self.conv1(x, edge_index).relu()
            h = self.conv2(h, edge_index)
            return h
        else:
            h = self.fc1(x).relu()
            h = self.fc2(h)
            return h

def compute_weights_from_embeddings(H, tau=0.5):
    with torch.no_grad():
        sim = (H @ H.t()) / max(1e-6, tau)
        W = torch.exp(sim)
        W = W / (W.sum(dim=1, keepdim=True) + 1e-8)
    return W


In [6]:

# 8.3 Ridge-WLS untuk koefisien lokal
def ridge_wls_local_beta(X, y, W_row, lam=0.1):
    M, p = X.shape
    w = W_row.reshape(-1,1)
    Xw = X * np.sqrt(w)
    yw = y * np.sqrt(w)
    A = Xw.T @ Xw + lam * np.eye(p)
    b = Xw.T @ yw
    try:
        beta = np.linalg.solve(A, b)
    except np.linalg.LinAlgError:
        beta = np.linalg.pinv(A) @ b
    return beta  # [p,1]

def estimate_all_betas(X_all, y_all, W, lam=0.1):
    T, N, p = X_all.shape
    TN = T*N
    beta_hat = np.zeros((T, N, p), dtype=np.float32)
    X_flat = X_all.reshape(TN, p)
    y_flat = y_all.reshape(TN, 1)
    W_np = W.cpu().numpy()
    for idx in range(TN):
        beta = ridge_wls_local_beta(X_flat, y_flat, W_np[idx], lam=lam)
        t = idx // N; i = idx % N
        beta_hat[t, i, :] = beta.flatten()
    return beta_hat


In [7]:

# 8.4 Training semi-supervised (sederhana): optimize GNN embeddings to minimize MSE via GTWR-GNN predictions
def train_gtnnwr(X_all, y_all, coords_blocks, k_spatial=8, use_temporal=True, lam=0.1, tau=0.5,
                 hidden_dim=64, out_dim=32, use_gat=False, epochs=10, lr=1e-2, train_ratio=0.7, val_ratio=0.15):
    T, N, p = X_all.shape
    TN = T*N
    X_flat = X_all.reshape(TN, p).astype(np.float32)
    y_flat = y_all.reshape(TN, 1).astype(np.float32)

    # Split mask
    idx = np.arange(TN)
    rng = np.random.default_rng(42)
    rng.shuffle(idx)
    n_tr = int(train_ratio*TN); n_va = int(val_ratio*TN)
    tr_idx = idx[:n_tr]; va_idx = idx[n_tr:n_tr+n_va]; te_idx = idx[n_tr+n_va:]

    # Build graph
    edge_index_np = build_spatiotemporal_graph(coords_blocks, k_spatial=k_spatial, use_temporal=use_temporal)
    edge_index = torch.tensor(edge_index_np, dtype=torch.long, device=DEVICE)

    # Model
    gnn = SimpleGCN(in_dim=p, hidden_dim=hidden_dim, out_dim=out_dim, use_gat=use_gat).to(DEVICE)
    opt = torch.optim.Adam(gnn.parameters(), lr=lr)

    X_t = torch.tensor(X_flat, dtype=torch.float32, device=DEVICE)
    y_t = torch.tensor(y_flat, dtype=torch.float32, device=DEVICE)

    for ep in range(1, epochs+1):
        gnn.train()
        opt.zero_grad()
        if HAS_PYG:
            H = gnn(X_t, edge_index)
        else:
            H = gnn(X_t)
        W = compute_weights_from_embeddings(H, tau=tau)  # [TN, TN]
        # beta via ridge-WLS (numpy), detach to cpu first
        beta_hat = estimate_all_betas(X_all.astype(np.float32), y_all.astype(np.float32), W.detach().cpu(), lam=lam)
        y_hat = np.einsum("tnp,tnp->tn", beta_hat, X_all).reshape(TN, 1)
        y_hat_t = torch.tensor(y_hat, dtype=torch.float32, device=DEVICE)

        # Loss pada train index
        loss = F.mse_loss(y_hat_t[tr_idx], y_t[tr_idx])
        loss.backward()
        opt.step()

        # Val
        with torch.no_grad():
            val_mse = F.mse_loss(y_hat_t[va_idx], y_t[va_idx]).item()
            tr_mse = loss.item()
        if ep % max(1, epochs//5) == 0:
            print(f"[Epoch {ep:03d}] Train MSE={tr_mse:.4f} | Val MSE={val_mse:.4f}")

    # Final eval
    with torch.no_grad():
        if HAS_PYG:
            H = gnn(X_t, edge_index)
        else:
            H = gnn(X_t)
        W = compute_weights_from_embeddings(H, tau=tau)
        beta_hat = estimate_all_betas(X_all.astype(np.float32), y_all.astype(np.float32), W.detach().cpu(), lam=lam)
        y_hat = np.einsum("tnp,tnp->tn", beta_hat, X_all).reshape(TN, 1)
        rmse = float(np.sqrt(mean_squared_error(y_flat, y_hat)))
        mae = float(mean_absolute_error(y_flat, y_hat))
        r2 = float(r2_score(y_flat, y_hat))
    return {"beta_hat": beta_hat, "y_hat": y_hat.reshape(T, N, 1), "rmse": rmse, "mae": mae, "r2": r2}


In [8]:

# 8.5 Pipeline end-to-end
def run_end_to_end():
    df = load_or_make_dataframe(PATH_XLSX)
    times_sorted = sorted(df[TIME_COL].unique())
    P = build_panel_arrays(df, TIME_COL, TARGET_COL, FEATURE_COLS, LAT_COL, LON_COL, times_sorted)
    X_all, y_all = P["X_all"], P["y_all"]
    coords_blocks = P["coords_blocks"]
    print("Shapes: X_all", X_all.shape, "y_all", y_all.shape, "coords", coords_blocks.shape)

    # Standarisasi fitur per seluruh node
    T, N, p = X_all.shape
    scaler = StandardScaler()
    X_flat = X_all.reshape(T*N, p)
    X_flat = scaler.fit_transform(X_flat).astype(np.float32)
    X_all_std = X_flat.reshape(T, N, p)

    # Training GNN + WLS
    result = train_gtnnwr(X_all_std, y_all, coords_blocks, k_spatial=K_SPATIAL, use_temporal=USE_TEMPORAL,
                          lam=RIDGE_LAMBDA, tau=0.5, hidden_dim=64, out_dim=32, use_gat=False,
                          epochs=8, lr=1e-2, train_ratio=0.7, val_ratio=0.15)
    print("Final Metrics:", {k: result[k] for k in ["rmse","mae","r2"]})
    return result


In [9]:

# 8.6 Jalankan (gunakan synthetic jika file tidak ada)
# result = run_end_to_end()
# result["r2"]


## **9. Hasil, Diskusi, & Interpretasi**
- Koefisien lokal (beta_hat) memperlihatkan variasi efek kovariat lintas lokasi-waktu.
- Bobot W hasil GNN merepresentasikan kedekatan efektif, tidak terbatas jarak Euclidean.
- Bandingkan dengan GTWR kernel tetap untuk melihat peningkatan adaptivitas dan metrik kesalahan.

## **10. Referensi**
- Hastie & Tibshirani (1993) — Varying-Coefficient Models (JRSS-B)
- Fan & Zhang (2008) — Statistical Methods with Varying Coefficient Models (Springer)
- GNNWR/GTNNWR — konsep NN untuk pembobotan (akses 2025-09-30)
- SRGCNN — Spatial Regression GCN (GeoInformatica)
- GSTRGCN — Generalized Spatial–Temporal Regression GCN + Transformer (Complex & Intelligent Systems)